In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import selenium as se
import time
from webcrawler.researchgate import Crawler
import mysql.connector
import textdistance
import xmltodict

In [2]:
cnx = mysql.connector.connect(user='david', password='daviddung1993',
                              host='127.0.0.1',
                              database='computervision')
cursor = cnx.cursor()

In [8]:
cursor.execute("select distinct p.PaperID, p.Title from Papers p, authoredBy b, Authors a where p.PaperID not like '1111%' and p.DOI like '10.1117%' and p.PaperID = b.PaperID and b.AuthoredByID not in (select a.AuthorID from affiliatedTo a) and a.AuthorID = b.AuthoredByID group by p.PaperID, p.Title, p.DOI order by p.PaperID DESC")
all_paper = cursor.fetchall()
len(all_paper)

NameError: name 'cursor' is not defined

In [6]:
cursor.execute("select distinct p.PaperID, p.Title from Papers p where p.journalName like '%ieee%'")
all_paper2 = cursor.fetchall()
len(all_paper2)

17246

In [4]:
for idx, paper in enumerate(all_paper):
    if idx % 10 == 0:
        print(idx)
    title = paper[1]

    missing_affiliation = False

    cursor.execute("select distinct a.AuthorID, a.Name from Papers p,authoredBy b,Authors a where p.PaperID=%s and p.PaperID = b.PaperID and b.AuthoredByID not in (SELECT AuthorID from affiliatedTo) and b.AuthoredByID = a.AuthorID",[paper[0]])
    authors = cursor.fetchall()

    if not authors or authors[0][0] == "0":
        continue

    try:
        first_name = authors[0][1].split()[-1]
    except:
        print(authors)
        raise ValueError

    #affiliations = ieee_crawler.make_request(title, first_name)
    try:
        affiliations, doi = research_crawler.make_request(title,first_name)
    except:
        print(paper[0])
        continue
    cursor.execute("UPDATE Papers p SET p.DOI=%s where p.PaperID=%s",[doi,paper[0]])

    if not affiliations:
        continue

    for author in authors:
        try:
            jaccard_dist = [textdistance.jaccard(author[1].lower().split(), aff_auth[0].lower().split()) for aff_auth in affiliations]
            if max(jaccard_dist) == 0:
                continue
            max_similarity_index = jaccard_dist.index(max(jaccard_dist))

            affiliation = affiliations[max_similarity_index][1]
            if not affiliations[max_similarity_index][1]:
                missing_affiliation = True
                continue
            cursor.execute("SELECT * FROM Affiliations a WHERE a.Name=%s", [affiliation])
            entry = cursor.fetchone()
            if not entry:
                cursor.execute("INSERT INTO Affiliations (Name) VALUES (%s)", [affiliation[:500]])
                lastrowid = cursor.lastrowid
            else:
                lastrowid = entry[0]
            cursor.execute("INSERT INTO affiliatedTo VALUES (%s,%s)", [author[0],lastrowid])
        except AttributeError:
            continue

    #print(f"DONE")
    #time.sleep(1)

    cnx.commit()

0
fe99e98984b3c61ffe49ef249f985648cb2bf101
fe3f0d65f2c3b3ae4f778b5a4f79f7d82a9fb8d1
fdde4309b1e46f057a39a61392bb09ebb4a67389
fd71ee88eb6c0644a937ed60bc52ba2b975475d3
fc8339ec19aa29e5e5b79f2105413dd8e58a311e
fc52c657a46973ae5e6f558e004389c170cee377
fbe4e6f67d1b00586dbe58cf638d551d6b66662f
fb7064747dd002c42f00826e5ce6257f98bda76c
fa55c99774dbd96a4d5753495c2a915d237a8c7f
f9da72bcb1da116a9b0f98c41c134f47407df5e4
10
f8c6a13580804e72ad7efd0b5dc50f4eb830c137
f881cea0594a562fa24164a6ece9db0774a3616f
f870421f3ad81885be624ddc22f8ac0b351dc85c
f86de8811afc32d664690f850837e691a3a28476
f86079dc1aa349dc5a834db051903a992b6c8d8d
f740a4d2286d7b0461afdd1c5a70930c2ef70210
f7100e9fd9b87c7d8cff9fea986254afadb392ea
f64b6f61cb0c28c17a113bada4b777127edadea1
f626a315ca955519a87d9b11ba7f31ce8bb92562
f5eb6f83ae33c0d1879867789a65e5d1afbd2f5f
20
f592561f0026a368fb17ffa53f3c09004b893382
f424eb49db06d3f6aa3a8679305a002aacde7a2c
f40936a796f381218728cf323777f29b84e431ff
f1fdc7c336891b166566a9ad18f1c9c3cc27282a
f19481af

In [5]:
headers = {"x-api-key": "M7HSjQNeTfai6l7JUiDZB8XYc85BHnHt3R0NXSEd"}
for idx, paper in enumerate(all_paper):
    if idx % 10 == 0:
        print(idx)
        cnx.commit()
    paper_link = f"https://api.semanticscholar.org/graph/v1/paper/{paper[0]}?fields=externalIds"
    doi = "NA"
    response = requests.get(paper_link, headers=headers).json()
    try:
        if "DOI" in response["externalIds"]:
            doi = response["externalIds"]["DOI"]
        elif "PubMed" in response["externalIds"]:
            doi = response["externalIds"]["PubMed"]
    except KeyError:
        pass
    cursor.execute("UPDATE Papers p SET p.DOI=%s where p.PaperID=%s",[doi,paper[0]])

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580


In [ ]:
classes = []
for element in soup.find_all(class_=True):
    classes.extend(element["class"])

In [ ]:
soup["header"]

In [ ]:
options = se.webdriver.ChromeOptions()
#options.add_argument('headless')
options.add_argument(
    "user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1.1 Safari/605.1.15")
browser = se.webdriver.Chrome(options=options)
browser.get("https://ieeexplore.ieee.org/document/9872831/authors#authors")
source = browser.page_source

In [ ]:
soup = BeautifulSoup(source)

In [ ]:
for node in soup.find_all("div", {"class":"authors-accordion-container"}):
    #print(node.findAll(Text=True))
    print(node.findAll(text=True))

In [ ]:
cursor.execute("select distinct a.AuthorID, a.Name from Papers p,authoredBy b,Authors a where p.PaperID=%s and p.PaperID = b.PaperID and b.AuthoredByID not in (SELECT AuthorID from affiliatedTo) and b.AuthoredByID = a.AuthorID",["00388ed29b7e6882bdf66b7b963cd8231d69ec75"])
authors = cursor.fetchall()
first_name = authors[0][1].split()[-1]
affiliations = research_crawler.make_request("Evidence for horizontal gene transfer in Escherichia coli speciation.",first_name)

In [ ]:
research_crawler.browser.quit()

In [ ]:
cursor.fetchall()

In [ ]:
cursor.close()
cnx.close()

In [ ]:
textdistance.jaccard("optimal combinations of pattern classifiers".split(), "optimal combination of pattern classifiers".split())

In [ ]:
options = se.webdriver.ChromeOptions()
options.add_argument('headless')

options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1.1 Safari/605.1.15")
browser = se.webdriver.Chrome(options=options)

In [ ]:
#url = "https://ieeexplore.ieee.org/search/searchresult.jsp?action=search&newsearch=true&matchBoolean=true&queryText=(%22All%20Metadata%22:Automatic%20Image%20Segmentation%20by%20Dynamic%20Region%20Merging)%20AND%20(%22Authors%22:Zhang)"
url = "https://www.researchgate.net/search/publication?q=machine learning"

browser.get(url)
#time.sleep(5)
html = browser.page_source
soup = BeautifulSoup(html)

In [ ]:
titles = []
authors = []
publications = []
for node in soup.find_all("div", {"class": "nova-legacy-v-publication-item__title"}):
    titles.append(''.join(node.findAll(text=True)))

for node in soup.find_all("ul", {"class": "nova-legacy-v-publication-item__person-list"}):
    authors.append(node.findAll(text=True))

for node in soup.find_all("a", {"class": "nova-legacy-e-link--theme-bare"}):
    publications.append(node["href"])

In [ ]:
alla = list(zip(titles,authors))
alla, publications

In [ ]:
link = f"https://www.researchgate.net/{publications[0]}"
browser.get(link)
html = browser.page_source
soup_author = BeautifulSoup(html)

In [ ]:
link

In [ ]:
authors_new = []
for node in soup_author.find_all("div", {"class": "nova-legacy-v-person-list-item__body"}):
    author_name = node.find_all("div", {"class":"nova-legacy-v-person-list-item__title"})[0].find(text=True)
    aff_element = node.find_all("li",{"class","nova-legacy-v-person-list-item__meta-item"})
    if aff_element:
        author_aff = aff_element[0].find(text=True)
    else:
        author_aff = ""
    authors_new.append((author_name, author_aff))


In [ ]:
authors_new

In [ ]:
authors_new

In [ ]:
import random
random.random() + 1

In [ ]:
jaccard_dista = [textdistance.jaccard("David Haeger".lower().split(), aff_auth.lower().split()) for aff_auth in ["David J. Fleet", "Hermann Wagner", "David J. Haeger"]]

In [ ]:
jaccard_dista